# Data Cleaning

---
Notebook ini adalah program untuk cleaning scraping data, clean data akan digunakan untuk proses analisis dan pemodelan
---

# Import Libaries

In [274]:
# Import Libraries
import json
import pandas as pd
import numpy as np
import re
import ast
from langdetect import detect, DetectorFactory
import warnings
warnings.filterwarnings('ignore')


# Load Data

menggabungkan semua json menjadi satu dataframe

In [275]:
# Read file JSON
with (open('./scrapping_data/adidas.json') as f1,
      open('./scrapping_data/nb.json') as f2,
      open('./scrapping_data/nike.json') as f3,
      open('./scrapping_data/puma.json') as f4,
      open('./scrapping_data/reebok.json') as f5):
    data1 = json.load(f1)
    data2 = json.load(f2)
    data3 = json.load(f3)
    data4 = json.load(f4)
    data5 = json.load(f5)

# Combine
combine = data1 + data2 + data3 + data4 + data5
# Save as backup
with open('all.json', 'w') as outfile:
    json.dump(combine, outfile, indent=2, ensure_ascii=True)

In [276]:
# Read file JSON
with open('all.json', 'r') as file:
    data = json.load(file)

# To DataFrame
df = pd.DataFrame(data)

# Show DataFrame
df


title              rating  \
0                     adidas Men's Run Falcon 5 Sneaker  4.4 out of 5 stars   
1                      adidas Men's Ultraboost 1.0 Shoe  4.4 out of 5 stars   
2                  adidas Women's Run Falcon 5 Sneakers  4.5 out of 5 stars   
3                          adidas Men's Kaptir 3.0 Shoe  4.5 out of 5 stars   
4                   adidas Men's Ultradream DNA Sneaker  4.5 out of 5 stars   
..                                                  ...                 ...   
730                   Reebok Classic Nylon (Little Kid)  4.6 out of 5 stars   
731           Reebok Women's Club C Grounds UK Sneakers  4.4 out of 5 stars   
732             Reebok Men's Club Memt Gum Classic Shoe  4.5 out of 5 stars   
733       Reebok Women's Club C Double Revenge Sneakers  4.6 out of 5 stars   
734  Reebok Work Zig Pulse Work Sneaker CT Men's Oxford  3.8 out of 5 stars   

    price  \
0     48.   
1    179.   
2     54.   
3     56.   
4     66.   
..    ...   
730   41.   
731   47.   
732   29.   
733   65.   
734   49.   

                                                                              img_url  \
0       https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1    https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2       https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3    https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4    https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   
..                                                                                ...   
730     https://m.media-amazon.com/images/I/71tgmXYfUxL._AC_SY395_SX395_QL70_ML2_.jpg   
731     https://m.media-amazon.com/images/I/51brEN-fGjL._AC_SY395_SX395_QL70_ML2_.jpg   
732  https://m.media-amazon.com/images/I/71TiRfHobkL._AC_SY395_SX395_QL70_FMwebp_.jpg   
733  https://m.media-amazon.com/images/I/7138dLkGonL._AC_SY395_SX395_QL70_FMwebp_.jpg   
734  https://m.media-amazon.com/images/I/61XU6NEQ2KL._AC_SY395_SX395_QL70_FMwebp_.jpg   

      Brand  \
0    Adidas   
1    Adidas   
2    Adidas   
3    Adidas   
4    Adidas   
..      ...   
730  Reebok   
731  Reebok   
732  Reebok   
733  Reebok   
734  Reebok   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            Customer_say  \
0                                                                                                                                                                                                                          Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                                                                                                                               Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, wit

In [277]:
# Save to CSV as backup
df.to_csv('data_before_clean.csv', index=False)

In [278]:
# Data Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         735 non-null    object
 1   rating        735 non-null    object
 2   price         735 non-null    object
 3   img_url       735 non-null    object
 4   Brand         735 non-null    object
 5   Customer_say  589 non-null    object
 6   Review        735 non-null    object
dtypes: object(7)
memory usage: 40.3+ KB


In [279]:
# Change Column Name
df.rename(columns={
    'title': 'product',
    'rating': 'rating',
    'price': 'price',
    'img_url': 'img_url',
    'Brand': 'brand',
    'Customer_say': 'customer_say',
    'Review': 'review'
}, inplace=True)

df.head()


product              rating price  \
0     adidas Men's Run Falcon 5 Sneaker  4.4 out of 5 stars   48.   
1      adidas Men's Ultraboost 1.0 Shoe  4.4 out of 5 stars  179.   
2  adidas Women's Run Falcon 5 Sneakers  4.5 out of 5 stars   54.   
3          adidas Men's Kaptir 3.0 Shoe  4.5 out of 5 stars   56.   
4   adidas Men's Ultradream DNA Sneaker  4.5 out of 5 stars   66.   

                                                                            img_url  \
0     https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1  https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2     https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3  https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4  https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   

    brand  \
0  Adidas   
1  Adidas   
2  Adidas   
3  Adidas   
4  Adidas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     customer_say  \
0                                                                                                                   Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                        Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, with some finding them soft while others say the soles are very hard.   
2                                                                                                                                                                              Customers find these running shoes comfortable, well-made, and appreciate their great arch support, with one customer noting they can stand all day without pain. They are lightweight, stylish, and offer good value for money. Customers disagree on the sizing, with some noting they run bigger than expected.   
3                                                                                                                          Customers find these sneakers comfortable and well-made, with a roomy fit that accommodates wide feet. They appreciate the look, value for money, and lightweight design. The durability receives mixed feedback - while some find them durable, others report they fall apart before wearing out. Customers disagree on the sizing, with some finding them too small.   
4  Customers find these sneakers comfortable, well-fitting, and good value for money, with one customer noting the wider toe box and another mentioning the flexible fit around the foot. The shoes receive positive feedback for their appearance and lightweight design, and customers appreciate their quality as walking shoes. The support and durability aspects receive mixed reviews - while some customers find them excellent, others report issues with holes developing in the shoes.   

                                                                                                        

# Create gender column and remove kid shoes

In [280]:
# Remove products for kids
df = df[~df['product'].str.contains('Kid', case=False, na=False)]

# Create gender column
df['gender'] = df['product'].apply(
    lambda x: (
        'Men' if 'Men' in x or "Men's" in x or 'Mens' in x
        else 'Women' if 'Women' in x or "Women's" in x or 'Womens' in x
        else 'Unisex'
    )
)

# Preview result
df.head()

product              rating price  \
0     adidas Men's Run Falcon 5 Sneaker  4.4 out of 5 stars   48.   
1      adidas Men's Ultraboost 1.0 Shoe  4.4 out of 5 stars  179.   
2  adidas Women's Run Falcon 5 Sneakers  4.5 out of 5 stars   54.   
3          adidas Men's Kaptir 3.0 Shoe  4.5 out of 5 stars   56.   
4   adidas Men's Ultradream DNA Sneaker  4.5 out of 5 stars   66.   

                                                                            img_url  \
0     https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1  https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2     https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3  https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4  https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   

    brand  \
0  Adidas   
1  Adidas   
2  Adidas   
3  Adidas   
4  Adidas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     customer_say  \
0                                                                                                                   Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                        Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, with some finding them soft while others say the soles are very hard.   
2                                                                                                                                                                              Customers find these running shoes comfortable, well-made, and appreciate their great arch support, with one customer noting they can stand all day without pain. They are lightweight, stylish, and offer good value for money. Customers disagree on the sizing, with some noting they run bigger than expected.   
3                                                                                                                          Customers find these sneakers comfortable and well-made, with a roomy fit that accommodates wide feet. They appreciate the look, value for money, and lightweight design. The durability receives mixed feedback - while some find them durable, others report they fall apart before wearing out. Customers disagree on the sizing, with some finding them too small.   
4  Customers find these sneakers comfortable, well-fitting, and good value for money, with one customer noting the wider toe box and another mentioning the flexible fit around the foot. The shoes receive positive feedback for their appearance and lightweight design, and customers appreciate their quality as walking shoes. The support and durability aspects receive mixed reviews - while some customers find them excellent, others report issues with holes developing in the shoes.   

                                                                                                        

In [281]:
# Check brand
df.brand.unique()

array(['Adidas', 'New Balance', 'Nike', 'PUMA', 'Reebok'], dtype=object)

In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 681 entries, 0 to 734
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product       681 non-null    object
 1   rating        681 non-null    object
 2   price         681 non-null    object
 3   img_url       681 non-null    object
 4   brand         681 non-null    object
 5   customer_say  541 non-null    object
 6   review        681 non-null    object
 7   gender        681 non-null    object
dtypes: object(8)
memory usage: 47.9+ KB


# Data Cleaning

## Column product

drop missing value

In [283]:
# Change 'N/A' to NaN
df['product'] = df['product'].replace('N/A', np.nan)

In [284]:
# Drop Data null in products
df = df.dropna(subset=['product']).reset_index(drop=True)

memastikan ada brand di nama produk

In [285]:
# Filter products based on Brand

# Create regex pattern with keyword
pattern = r'^(Adidas|New Balance|Nike|PUMA|Reebok)'

# Filter products with keyword product (case-insensitive)
df = df[df['product'].str.contains(pattern, case=False, na=False, regex=True)]

# Reset index
df = df.reset_index(drop=True)


In [286]:
# Make sure brand is in product name
df = df[df.apply(lambda row: str(row['brand']).lower() in str(row['product']).lower(), axis=1)]

Memastikan data brand tercantum di kolom nama produk

In [287]:
df['product'].unique()

array(["adidas Men's Run Falcon 5 Sneaker",
       "adidas Men's Ultraboost 1.0 Shoe",
       "adidas Women's Run Falcon 5 Sneakers",
       "adidas Men's Kaptir 3.0 Shoe",
       "adidas Men's Ultradream DNA Sneaker",
       "adidas Men's Duramo Speed 2 Running Sneaker",
       "adidas Men's Racer TR23 Sneaker",
       "adidas Men's Lite Racer 3.0 Running Shoe",
       "adidas Men's Run Falcon 3.0 Shoe",
       "adidas Men's Kaptir 4.0 Sneaker",
       "adidas Men's X_PLR Path Sneaker",
       "adidas Men's Lightblaze Sneaker",
       "adidas Women's Response Super Sneaker",
       'adidas Womens X_PLR Path', "adidas Men's Run 72 Sneaker",
       "adidas Men's Run 84 Shoes Sneaker",
       "Adidas Men's Lightshift Sneaker",
       "adidas Men's Supernova Ease Running Sneaker",
       "adidas Men's Terrex Tracefinder Trail Running Sneaker",
       "adidas Women's Kaptir Flow Sneaker",
       "adidas Men's Ultrarun 5 Sneaker",
       "adidas Women's Run Falcon 5 Trail Sneaker",
       "

Menghapus setelah koma, (dalam kurung), dan kata kata seperti trail, running, and shoe yang berada pada nama produk.

In [288]:
# Combined cleaning pattern
pattern_words = r'\b(trail|running|shoe|shoes|sneaker|sneakers)\b'

df['product'] = (
    df['product']
    .str.replace(r'\([^)]*\)', '', regex=True)   # remove (...) and contents
    .str.replace(r',.*', '', regex=True)         # remove comma and everything after
    .str.replace(pattern_words, '', flags=re.IGNORECASE, regex=True)  # remove keywords
    .str.replace(r'[^A-Za-z0-9\s]', '', regex=True)  # ❗ remove all symbols
    .str.replace(r'\s+', ' ', regex=True)        # normalize spaces
    .str.strip()
)



In [289]:
df['product'].unique()

array(['adidas Mens Run Falcon 5', 'adidas Mens Ultraboost 10',
       'adidas Womens Run Falcon 5', 'adidas Mens Kaptir 30',
       'adidas Mens Ultradream DNA', 'adidas Mens Duramo Speed 2',
       'adidas Mens Racer TR23', 'adidas Mens Lite Racer 30',
       'adidas Mens Run Falcon 30', 'adidas Mens Kaptir 40',
       'adidas Mens XPLR Path', 'adidas Mens Lightblaze',
       'adidas Womens Response Super', 'adidas Womens XPLR Path',
       'adidas Mens Run 72', 'adidas Mens Run 84',
       'Adidas Mens Lightshift', 'adidas Mens Supernova Ease',
       'adidas Mens Terrex Tracefinder', 'adidas Womens Kaptir Flow',
       'adidas Mens Ultrarun 5', 'adidas Womens Puremotion Adapt',
       'adidas Mens Supernova Prima', 'adidas Mens Astrastar',
       'adidas Mens', 'adidas Mens Ultraboost Light',
       'Adidas Mens Run Falcon 5', 'adidas Mens Ultrabounce 5',
       'adidas Womens Cloudfoam Pure', 'Adidas Womens Swift Run 22',
       'adidas Womens Run 72', 'adidas Mens Ultraboost 22',

## Column rating

Kolom rating perlu dilakukan striping setelah angka rating dan hasil diubah menjadi numerikal.

In [290]:
# Clean and Convert rating
df['rating'] = df['rating'].str.extract(r'(\d+\.?\d*)').astype(float)

In [291]:
df.head()

product  rating price  \
0    adidas Mens Run Falcon 5     4.4   48.   
1   adidas Mens Ultraboost 10     4.4  179.   
2  adidas Womens Run Falcon 5     4.5   54.   
3       adidas Mens Kaptir 30     4.5   56.   
4  adidas Mens Ultradream DNA     4.5   66.   

                                                                            img_url  \
0     https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1  https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2     https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3  https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4  https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   

    brand  \
0  Adidas   
1  Adidas   
2  Adidas   
3  Adidas   
4  Adidas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     customer_say  \
0                                                                                                                   Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                        Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, with some finding them soft while others say the soles are very hard.   
2                                                                                                                                                                              Customers find these running shoes comfortable, well-made, and appreciate their great arch support, with one customer noting they can stand all day without pain. They are lightweight, stylish, and offer good value for money. Customers disagree on the sizing, with some noting they run bigger than expected.   
3                                                                                                                          Customers find these sneakers comfortable and well-made, with a roomy fit that accommodates wide feet. They appreciate the look, value for money, and lightweight design. The durability receives mixed feedback - while some find them durable, others report they fall apart before wearing out. Customers disagree on the sizing, with some finding them too small.   
4  Customers find these sneakers comfortable, well-fitting, and good value for money, with one customer noting the wider toe box and another mentioning the flexible fit around the foot. The shoes receive positive feedback for their appearance and lightweight design, and customers appreciate their quality as walking shoes. The support and durability aspects receive mixed reviews - while some customers find them excellent, others report issues with holes developing in the shoes.   

                                                                                                                                                                                                                                  

In [292]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    object 
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  517 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(1), object(7)
memory usage: 46.1+ KB


## Column price

kolom harga akan ditambahkan 1 dollar karena ada nilai sen yang tidak di scrap, tipenya juga akan diubah

In [293]:
# Change 'N/A' to NaN
df['price'] = df['price'].replace('N/A', np.nan)

In [294]:
# Convert to numeric (float), ignoring non-numeric characters
df['price'] = df['price'].str.replace('[^\d.]', '', regex=True).astype(float)

# Add 1
df['price'] = df['price'] + 1

In [295]:
df.head()

product  rating  price  \
0    adidas Mens Run Falcon 5     4.4   49.0   
1   adidas Mens Ultraboost 10     4.4  180.0   
2  adidas Womens Run Falcon 5     4.5   55.0   
3       adidas Mens Kaptir 30     4.5   57.0   
4  adidas Mens Ultradream DNA     4.5   67.0   

                                                                            img_url  \
0     https://m.media-amazon.com/images/I/71DZ2p2173L._AC_SX395_SY395_QL70_ML2_.jpg   
1  https://m.media-amazon.com/images/I/71ljZ6ZzfxL._AC_SX395_SY395_QL70_FMwebp_.jpg   
2     https://m.media-amazon.com/images/I/71v0+G-P0uL._AC_SX395_SY395_QL70_ML2_.jpg   
3  https://m.media-amazon.com/images/I/71jMV8lsi-L._AC_SY395_SX395_QL70_FMwebp_.jpg   
4  https://m.media-amazon.com/images/I/71UmLJnEA0L._AC_SY395_SX395_QL70_FMwebp_.jpg   

    brand  \
0  Adidas   
1  Adidas   
2  Adidas   
3  Adidas   
4  Adidas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     customer_say  \
0                                                                                                                   Customers find these running shoes comfortable, well-made, and good value for money, with great arch support and a nice look. The fit and sizing receive mixed feedback - while some find them perfect, others note they run narrow and are not true wide shoes. Durability is also mixed, with some saying they hold up well while others report them falling apart quickly.   
1                        Customers find these running shoes comfortable, well-made, and nice-looking, offering great value for money. The fit and sizing receive mixed feedback - while some find them a good fit, others note they run very small and are very narrow. The durability is also mixed, with some saying they last a long time while others report them falling apart. The cushioning receives mixed reviews, with some finding them soft while others say the soles are very hard.   
2                                                                                                                                                                              Customers find these running shoes comfortable, well-made, and appreciate their great arch support, with one customer noting they can stand all day without pain. They are lightweight, stylish, and offer good value for money. Customers disagree on the sizing, with some noting they run bigger than expected.   
3                                                                                                                          Customers find these sneakers comfortable and well-made, with a roomy fit that accommodates wide feet. They appreciate the look, value for money, and lightweight design. The durability receives mixed feedback - while some find them durable, others report they fall apart before wearing out. Customers disagree on the sizing, with some finding them too small.   
4  Customers find these sneakers comfortable, well-fitting, and good value for money, with one customer noting the wider toe box and another mentioning the flexible fit around the foot. The shoes receive positive feedback for their appearance and lightweight design, and customers appreciate their quality as walking shoes. The support and durability aspects receive mixed reviews - while some customers find them excellent, others report issues with holes developing in the shoes.   

                                                                                                                                                                                                                            

In [296]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    float64
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  517 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(2), object(6)
memory usage: 46.1+ KB


Kolom price sudah bersih dan berbentuk type numerical.

## Column img_url

In [297]:
# Change 'N/A' to NaN
df['img_url'] = df['img_url'].replace('N/A', np.nan)

In [298]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    float64
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  517 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(2), object(6)
memory usage: 46.1+ KB


# Handling Missing Value

In [299]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    float64
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  517 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(2), object(6)
memory usage: 46.1+ KB


handling missing value customer_say

In [300]:
df['customer_say'] = df['customer_say'].fillna("No summary available.")

In [301]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       655 non-null    object 
 1   rating        655 non-null    float64
 2   price         655 non-null    float64
 3   img_url       655 non-null    object 
 4   brand         655 non-null    object 
 5   customer_say  655 non-null    object 
 6   review        655 non-null    object 
 7   gender        655 non-null    object 
dtypes: float64(2), object(6)
memory usage: 46.1+ KB


# Handling duplicate

cek duplikasi nama produk, menggabungkan review dan summary untuk produk yang sama

In [302]:
def safe_join(values):
    result = []
    for v in values:
        if isinstance(v, list):
            result.extend(map(str, v))
        elif pd.notna(v):
            result.append(str(v))
    return ' '.join(result)

# Group by 'product' and merge all other columns
df_merged = (
    df
    .groupby('product', dropna=False)
    .agg(lambda x: safe_join(x))
    .reset_index()
)

In [303]:
dupes = df_merged[df_merged['product'].duplicated(keep=False)]
print(dupes['product'].unique())

[]


# Handling review

mengubah semua json review menjadi satu

In [304]:
# See full text (avoid misleading "...")
pd.set_option('display.max_colwidth', None)

import re

def robust_parse_plus(val):
    # Keep lists/dicts
    if isinstance(val, (list, dict)) or pd.isna(val):
        return val
    
    if isinstance(val, bytes):
        try:
            val = val.decode('utf-8', errors='ignore')
        except Exception:
            return val
    
    if isinstance(val, str):
        s = val.strip()
        # Try JSON
        try:
            return json.loads(s)
        except Exception:
            pass
        # Try Python literal
        try:
            return ast.literal_eval(s)
        except Exception:
            pass
        # Fallback: extract `'review': '...'` or `"review": "..."`
        m = re.search(r"(?:'review'|\"review\")\s*:\s*(?:'|\")(.*?)(?:'|\")", s)
        if m:
            return {'review': m.group(1)}
        # As a last resort, keep the raw string
        return s
    return val

def normalize_to_list_of_review_dicts_plus(val):
    if pd.isna(val):
        return np.nan
    if isinstance(val, dict):
        if 'review' not in val:
            for k in ('text','comment','content','body'):
                if k in val:
                    val = {'review': val[k], **{kk: vv for kk, vv in val.items() if kk != k}}
                    break
        return [val]
    if isinstance(val, Iterable) and not isinstance(val, (str, bytes)):
        out = []
        for item in val:
            if isinstance(item, dict):
                if 'review' not in item:
                    for k in ('text','comment','content','body'):
                        if k in item:
                            item = {'review': item[k], **{kk: vv for kk, vv in item.items() if kk != k}}
                            break
                out.append(item)
            elif isinstance(item, str) and item.strip():
                out.append({'review': item})
        return out or np.nan
    if isinstance(val, str) and val.strip():
        return [{'review': val}]
    return np.nan

# 1) Parse (stronger)
df_merged['review'] = df_merged['review'].apply(robust_parse_plus)

# 2) Normalize to list of dicts
df_merged['review'] = df_merged['review'].apply(normalize_to_list_of_review_dicts_plus)

# 3) Explode
df_exploded = df_merged.explode('review', ignore_index=True)

# 4) Extract review text and rating with robust fallback
def get_review_text(x):
    if isinstance(x, dict):
        for k in ('review','text','comment','content','body'):
            if k in x:
                return x[k]
    if isinstance(x, str):
        # handle raw strings that still look like dicts
        m = re.search(r"(?:'review'|\"review\")\s*:\s*(?:'|\")(.*?)(?:'|\")", x)
        return m.group(1) if m else x
    return np.nan

def get_rating_val(x):
    if isinstance(x, dict):
        for k in ('rating','stars','score'):
            if k in x:
                return x[k]
    if isinstance(x, str):
        m = re.search(r"(?:'rating'|\"rating\")\s*:\s*([0-9]+(?:\.[0-9]+)?)", x)
        return float(m.group(1)) if m else np.nan
    return np.nan

df_exploded['individual_review'] = df_exploded['review'].apply(get_review_text)

# Clean up whitespace and drop empties
df_exploded['individual_review'] = (
    df_exploded['individual_review']
      .astype('string')
      .str.replace(r'\s+', ' ', regex=True)
      .str.strip()
)

# Optional: remove the nested column
df_exploded = df_exploded.drop(columns=['review'])

# If you only want rows that actually have extracted text:
df_exploded = df_exploded[df_exploded['individual_review'].notna() & (df_exploded['individual_review'] != '')]


In [305]:
df_exploded

product rating price  \
0    Adidas Fortarun 20 Cloudfoam Lace    4.5  43.0   
1               Adidas Mens Lightblaze    5.0  27.0   
2               Adidas Mens Lightshift    4.8  40.0   
3             Adidas Mens Run Falcon 5    4.5  56.0   
4                Adidas Mens XPLR Path    4.7  60.0   
..                                 ...    ...   ...   
596        adidas Womens Ultraboost 23    4.4  92.0   
597      adidas Womens Ultradream Bold    4.5  71.0   
598           adidas Womens Ultrarun 5    4.4  59.0   
599            adidas Womens XPLR Path    4.4  60.0   
600           adidas Womens XPLR Phase    4.3  41.0   

                                                                              img_url  \
0    https://m.media-amazon.com/images/I/61BuM-r7P0L._AC_SX395_SY395_QL70_FMwebp_.jpg   
1       https://m.media-amazon.com/images/I/8195tyJbBvL._AC_SY395_SX395_QL70_ML2_.jpg   
2       https://m.media-amazon.com/images/I/7114rduvMKL._AC_SY395_SX395_QL70_ML2_.jpg   
3       https://m.media-amazon.com/images/I/71eXwqTa5BL._AC_SX395_SY395_QL70_ML2_.jpg   
4    https://m.media-amazon.com/images/I/71wGLUKuCJL._AC_SX395_SY395_QL70_FMwebp_.jpg   
..                                                                                ...   
596  https://m.media-amazon.com/images/I/71AKhXmrsYL._AC_SX395_SY395_QL70_FMwebp_.jpg   
597  https://m.media-amazon.com/images/I/71s7iwqOZGL._AC_SY395_SX395_QL70_FMwebp_.jpg   
598  https://m.media-amazon.com/images/I/71CcDlAHIcL._AC_SY395_SX395_QL70_FMwebp_.jpg   
599  https://m.media-amazon.com/images/I/71MFZKXCmcL._AC_SX395_SY395_QL70_FMwebp_.jpg   
600  https://m.media-amazon.com/images/I/71vr5YC6VOL._AC_SY395_SX395_QL70_FMwebp_.jpg   

      brand  \
0    Adidas   
1    Adidas   
2    Adidas   
3    Adidas   
4    Adidas   
..      ...   
596  Adidas   
597  Adidas   
598  Adidas   
599  Adidas   
600  Adidas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          customer_say  \
0                                                                                                     Customers find these running shoes to be well-made, comfortable, and easy for children to put on, with good durability and value for money. They appreciate the style, particularly the colors, and one customer mentions the nice wide toe box. The size receives mixed feedback - while some say they fit true to size, others find them too short. The velcro closure is problematic, with customers reporting that it doesn't stay in place.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                No summary available.   
2                                                                                                                                                                                                                                                                                                              Customers find these walking shoes comfortable and well-made, with a great fit and lightweight design. They appreciate the appearance, with one customer noting they don't look orthopedic, and consider th

memastikan bahasa yang digunakan adalah bahasa inggris

In [306]:
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 599 entries, 0 to 600
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product            599 non-null    object
 1   rating             599 non-null    object
 2   price              599 non-null    object
 3   img_url            599 non-null    object
 4   brand              599 non-null    object
 5   customer_say       599 non-null    object
 6   gender             599 non-null    object
 7   individual_review  599 non-null    string
dtypes: object(7), string(1)
memory usage: 42.1+ KB


In [307]:
# Keep individual_review in English only
DetectorFactory.seed = 8

def is_english(text):
    if not isinstance(text, str) or text.strip() == '':
        return False
    try:
        return detect(text) == 'en'
    except:
        return False

# apply filter
df_exploded['is_english'] = df_exploded['individual_review'].apply(is_english)

# Drop that is not English
df_en = df_exploded[df_exploded['is_english']].drop(columns=['is_english']).reset_index(drop=True)

In [308]:
# Clean and convert "rating" to float
df_en["rating"] = (
    df_en["rating"]
      .astype(str)
      .str.extract(r"([\d\.]+)")      # extract numeric part like 4.5
      .astype(float)
)

# Clean and convert "price" to int
df_en["price"] = (
    df_en["price"]
      .astype(str)
      .str.replace(r"[^0-9.]", "", regex=True)  # keep digits and dots
      .replace("", None)
)

df_en["price"] = pd.to_numeric(df_en["price"], errors="coerce")  # float64


In [309]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product            536 non-null    object 
 1   rating             536 non-null    float64
 2   price              496 non-null    float64
 3   img_url            536 non-null    object 
 4   brand              536 non-null    object 
 5   customer_say       536 non-null    object 
 6   gender             536 non-null    object 
 7   individual_review  536 non-null    string 
dtypes: float64(2), object(5), string(1)
memory usage: 33.6+ KB


In [310]:
# Check duplicate
df_en.duplicated().sum()

0

In [311]:
# Check if there is 'N/A'
print(df_en['individual_review'].str.contains('N/A', na=False).any())

False


In [312]:
# Save data_product
df_en.to_csv('data_clean.csv', index=False)